In [62]:
# Import needed packages
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
import csv
import sys
import pandas_datareader as pdr
import datetime
from nltk.corpus import stopwords
import nltk

In [93]:
startdate = datetime.datetime(2022,3,30)
enddate = datetime.datetime(2022,4,5)
st = pdr.get_data_yahoo('MMM',start=startdate,end=enddate)
st

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-03-30,153.259995,150.770004,151.330002,151.429993,2395100,151.429993
2022-03-31,151.250000,148.800003,150.449997,148.880005,3193800,148.880005
2022-04-01,149.860001,146.850006,149.630005,149.690002,2196400,149.690002
2022-04-04,150.009995,146.080002,149.059998,149.529999,2631800,149.529999
2022-04-05,150.630005,148.509995,148.649994,148.899994,2148600,148.899994


In [26]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
consumer_key = 'aR8jQP7cfr623h9apTvM711pm'
consumer_secret = 'SLKF8TOUL8PtapZ5amz4Y9ZBRE99ocRxwvEnzIjfEFri2TbhTI'
access_key= '1511751059332493317-eFiU4zDgdWqZPDC7EWZrQVaCLJ5ZX8'
access_secret = 'U3yAxDcXSFVHqsbJW1adpZiElFoIBosY7FW1UJpAOULqM'

#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

query = "$"+"MMM" + " -filter:retweets -filter:replies"
tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', since = startdate, until = enddate, tweet_mode='extended').items(5)
print(tweets)
for tweet in tweets:
    status=tweet._json
    created_at = status['created_at']
    tweet_id = status['id_str']
    tweet_text = status['full_text']
    tweet_text = re.sub(r'\W', ' ',tweet_text)
    print(created_at)

Unexpected parameter: since


Thu Mar 31 22:51:05 +0000 2022
Thu Mar 31 22:30:02 +0000 2022
Thu Mar 31 20:28:37 +0000 2022
Thu Mar 31 17:33:06 +0000 2022
Tue Mar 29 23:03:05 +0000 2022


In [90]:
# Obtain stock tickers

data = pd.read_html('https://en.wikipedia.org/wiki/List_of_S&P_500_companies')
#print(data)
table = data[0]
#table.head()
tickers = table['Symbol'].tolist()
tickers1 = tickers[:49]
tickers2 = tickers[49:99]
tickers3 = tickers[99:149]
tickers4 = tickers[149:199]
tickers5 = tickers[199:249]
tickers6 = tickers[249:299]
tickers7 = tickers[349:399]
tickers8 = tickers[449:]


In [4]:
# Get stock prices and returns
startdate = datetime.datetime(2022,3,30)
enddate = datetime.datetime(2022,4,5)

file = open("stocks.csv",'w',newline='')
print(len(tickers))
count = 0
for ticker in tickers:
    count +=1
    print(count)
    try:
        st = pdr.get_data_yahoo(ticker,start=startdate,end=enddate)
        st['pct_change'] = st['Adj Close'].pct_change(4)
        #print(ticker + str(st['Adj Close']) + str(st['pct_change']))
        stockdata = csv.writer(file)
        stockdata.writerow([ticker,st['pct_change'].iloc[-1]])
    except:
        pass



505
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [101]:
# Get Twitter data and do sentiment analysis
# Authenticate credentials

#Twitter credentials for the app
# consumer_key = 'ELRou1VKpS5p7EBEGamXqqGRT'
# consumer_secret = 'QbsToHJOyA9ZsDrFCCw1aErF9LubwqlfLpLcJviruhKepQwX3D'
# access_key= '3138259520-6jOClvoHXzF5DCf2XaylTxhrn1Rb7X50ykQyU3C'
# access_secret = 'r9bokM5UvRbS8QnsYdQksSi2tO8gRj7T4sfq2K41jzmQM'

# consumer_key = 'aR8jQP7cfr623h9apTvM711pm'
# consumer_secret = 'SLKF8TOUL8PtapZ5amz4Y9ZBRE99ocRxwvEnzIjfEFri2TbhTI'
# access_key= '1511751059332493317-eFiU4zDgdWqZPDC7EWZrQVaCLJ5ZX8'
# access_secret = 'U3yAxDcXSFVHqsbJW1adpZiElFoIBosY7FW1UJpAOULqM'

# consumer_key = 'Rd9nlZE2GWgkXXj6LfCGnWCtq'
# consumer_secret = 'yCQeUkiwCUtgsvdn46jQiK6yqISlXy8xlbV3PG4rr8sb3qgWpZ'
# access_key= '1432859916277063683-ZAXekHTSaIi3IecnfXAAlR1iCDWEO6'
# access_secret = 'LtxEkUYYuuRAePwCJaDhexTa2KwKn8nem26SZSHsfc2bu'

consumer_key = 'lLoOz2cbplDwX3vTzZFv2Hyqz'
consumer_secret = 'Yp5YFPvY3ISI2KUVQC6HAqYjDCUWw5na3IFrCvTVoTuvF9QddN'
access_key= '3028339548-i1TWGaqfItqlybCtkJZUQ7km95pylMZl8zPQBLb'
access_secret = 'llC8DaUitlD5wMl0FMqSxR3L78q0OaF56oazKJF5Hqnnr'





#Pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key,access_secret)
# api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify=True)
api = tweepy.API(auth)

file = open("searchTweets8.csv","w",newline='')
alltweets = csv.writer(file)
count = 0
for ticker in tickers8:
    count += 1
    print(count)
    query = "$"+ticker + " -filter:retweets -filter:replies"
    tweets = tweepy.Cursor(api.search_tweets ,q=query,lang='en', until = enddate, tweet_mode='extended').items(20)
    for tweet in tweets:
        status=tweet._json
        created_at = status['created_at']
        tweet_id = status['id_str']
        tweet_text = status['full_text']
        tweet_text = re.sub(r'\W', ' ',tweet_text)
        print(tweet_text)
        try: 
            alltweets.writerow([ticker,created_at,tweet_id, tweet_text])
        except UnicodeEncodeError:
            pass




1
 TMO Max Pain 580 00  Maturity 04 14 2022   maxpain  options https   t co 9lLeN10XC0 https   t co Q3urzpkiZb
Stock   TMO        REALTIME TREND     BUY BUY   95 00   SELL   5 00   Try now the Best Pivot Point Calculator   https   t co 3uJsHhr1jK  tradingview  stocks  stockstowatch
 TMO Strike  570 00 Expiration  2022 04 14 Put  TMO220414P570 Price  2 40 Size  3203 Time  2022 03 31 12 44 19PM Premium   768 720 00
2
 ANF short interest ratio is 4 47 on 2019 05 31 https   t co bsyppTQVpw  BKE 14 49  EXPR 8 00  TJX 1 92  GPS 4 53
   TJX price target update    Target Lowered by Barclays From  94 00 to  82 00  Day quote change   55  0 51   Target upside  49 09   Published  April 1  2022 https   t co 4nNP3BgAoz
   TJX price target update    Target Lowered by Barclays From  94 00 to  82 00  Day quote change   60 53   0 03   Target upside  35 47   Published  April 1  2022 https   t co 4nNP3BgAoz
 TJX   Canestrari Kenneth sold 94526 0 shares of Common Stock worth  1 444 228 00 in 3 transaction 

13
14
 ULTA  15s  delayed   Issued Press Release on March 31  16 05 00  Ulta Beauty to Participate in the J P  Morgan Retail Round Up Conference https   t co pYy41sZ01l
 ULTA Bearish Engulfing Daily appearances since 2010  141 Close 5 days later    Avg  1 30 SD  11 40 Worst   95 01 Best  41 41  10 days later  Avg  1 00 SD  14 85 Worst   95 62 Best  47 26  30 days later  Avg  2 87 SD  20 55 Worst   85 73 Best  55 11
15
   UAA price target update    Target Lowered by Robert W  Baird From  24 00 to  22 00  Day quote change   15 5   1 47   Target upside  41 94   Published  April 1  2022 https   t co OkSixjXscF
   UAA price target update    Target Lowered by Barclays From  24 00 to  22 00  Day quote change   15 5   1 47   Target upside  41 94   Published  April 1  2022 https   t co OkSixjXscF
   UAA price target update    Target Lowered by Robert W  Baird From  24 00 to  22 00  Day quote change   17 04  0 15   Target upside  29 11   Published  April 1  2022 https   t co OkSixjXscF
16
Nice p

 VRSN  15s  delayed   Issued Press Release on April 01  16 05 00  Verisign to Report First Quarter 2022 Financial Results https   t co sBH74dhm3c
26
 VRSK  15s  delayed   Issued Press Release on April 04  08 00 00  Verisk Assembles Climate Advisory Council https   t co 5TRQC4gYkV
Verisk Analytics Inc    VRSK    QUOTE  VRSK was DOWN   on Fri Apr 01 Closed at 214 12 USD for a 0 00  loss from previous close          ANALYSIS Verisk Analytics Inc should be nationalized      
 VRSK high OI range is 165 00 to 290 00 for option expiration 04 14 2022 PutCallRatio 0 38  maxpain  options https   t co ykRyRjAlhs https   t co AnbNVQeR7D
27
 VZ Strike  52 50 Expiration  2022 06 17 Put  VZ220617P52 5 Price  1 89 Size  3000 Time  2022 04 04 2 35 51PM Premium   567 000 00
 VZ  15s  delayed   Issued Press Release on April 04  10 22 00  Retirees Fight to Strengthen Clawback Policy  Limit Executive Golden Parachutes at Verizon https   t co QHsX6aSSSU
Nice print for  VZ Size  150000 Price  51 95 Amount   

 VNO Max Pain 45 00  Maturity 04 14 2022   maxpain  options https   t co 05yKgmA6aI https   t co jEurUw5LGQ
33
 VMC high OI range is 155 00 to 200 00 for option expiration 04 14 2022 PutCallRatio 1 26  maxpain  options https   t co gMhsWZSUJA https   t co Kdt4fngA5b
 VMC high OI range is 155 00 to 200 00 for option expiration 04 14 2022 PutCallRatio 1 25  maxpain  options https   t co gMhsWZSUJA https   t co n8iAdQ9IvI
34
35
 WMT Option to look at          150 00 Call for Thursday  April 14  2022  Roughly 397 Thousand dollars     Learn more on StockOrbit  Link on profile    
 WMT 05 20 22  160 00 PUT  Type  large Time  14 00 21 Total Size  5000 Contracts Total Value   5350000 Average Price   10 7 Ask   10 7 Bid   10 55 Underlying   150 73 Sentiment  bearish
Watchlist for 04 04  WMT  C  gt  151 50  P  lt  150 50  ZS  C  gt  250 00  P  lt  244 51  CRWD  C  gt  232 30  P  lt  223 44  OKTA  C  gt  150 17  P  lt  147 08  XOM  C  gt  84 29  P  lt  82 08  PYPL  C  gt  118 24  P  lt  115 03  A

Nice to see that my top 10 US bank fund is doing well today  tfc  2 32   wfc  2 38  bac   3 00   usb   2 26   pnc   1 94   bk   2 76   jpm 2 06   cof  1 42   tp   1 34   c 2 11 
 WFC Strike  52 00 Expiration  2022 04 14 Call  WFC220414C52 Price  0 64 Size  1119 Time  2022 03 31 10 40 25AM Premium   71 616 00
 WFC Strike  55 00 Expiration  2022 04 14 Call  WFC220414C55 Price  0 22 Size  10000 Time  2022 03 31 10 58 00AM Premium   220 000 00
 WFC Strike  55 00 Expiration  2022 04 14 Call  WFC220414C55 Price  0 19 Size  10000 Time  2022 03 31 10 40 15AM Premium   190 000 00
 WFC Strike  42 50 Expiration  2024 01 19 Put  WFC240119P42 5 Price  4 80 Size  2200 Time  2022 03 31 12 26 49PM Premium   1 056 000 00
41
 WELL  15s  delayed  filed form 424B5 on April 04  10 24 22 https   t co ocU5taXxmV
 WELL high OI range is 85 00 to 95 00 for option expiration 04 14 2022 PutCallRatio 2 35  maxpain  options https   t co xiksgQxQzt https   t co 6Am0ALfLVB
   WELL price target update    Target Raised

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

In [75]:
# Text Processing

# Open tweets CSV
file = open("searchTweets.csv","r")
tweets = pd.read_csv(file)

# Import stopwords with nltk.
stop = stopwords.words('english')

# Exclude stopwords with Python's list comprehension and pandas.DataFrame.apply.
tweets['tweet_without_stopwords'] = tweets['tweet'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Text Normalization
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    words = [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]
    return " ".join(words)
tweets['processed_tweet'] = tweets.tweet_without_stopwords.apply(lemmatize_text)


tweets[['processed_tweet']].values[0][0]


'Stock MMM REALTIME TREND BUY BUY 95 00 SELL 5 00 Try Best Pivot Point Calculator http co Q5ES63osKo tradingview stock stockstowatch'

In [76]:
# Sentiment analysis

file = open("searchTweets.csv","r")
alltweets = csv.reader(file)
print(alltweets)
sentiment_file = open("sentimenttweets.csv","w",newline='')
sntTweets=csv.writer(sentiment_file)
count = 0
for row in alltweets:
    processed_tweet = tweets[['processed_tweet']].values[count][0]
    blob = TextBlob(row[3])
    blob2 = TextBlob(processed_tweet)
    if blob2.sentiment.polarity>0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"positive", blob.sentiment.subjectivity, blob2.sentiment.polarity, blob2.subjectivity])
    elif blob2.sentiment.polarity<0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"negative", blob.sentiment.subjectivity, blob2.sentiment.polarity, blob2.subjectivity])    
    elif blob2.sentiment.polarity==0:
        sntTweets.writerow([row[0],row[1],row[2],blob.sentiment.polarity,"neutral", blob.sentiment.subjectivity, blob2.sentiment.polarity, blob2.subjectivity])
    count += 1
        
file.close()
sentiment_file.close()